# Deterministic Agent Workflow for Financial Services - Loan Application Processing

This notebook demonstrates a complete **deterministic agent pipeline** for automating a loan application decision process proposed by Team 5 (Full Code) as part of the **Oxford Artificial Intelligence Summit 2025: Autonomous AI Agents** capstone project. The workflow runs a fixed sequence of agent tools and combines their results into a final summary, which is then emailed using a Zapier workflow and MCP integration.

A deterministic workflow was chosen for this project to ensure transparency, reproducibility, and ease of auditing in the loan decision process. By following a clearly defined sequence of steps, the workflow guarantees that every application is evaluated consistently according to the same business rules, making it ideal for regulated environments where explainability and compliance are critical.

The two main sections of this notebook are: 
1. **Deterministic Workflow** — simple baseline flow with hardcoded loan application data
2. **Enhanced Flow** — calls the Zapier MCP server for composing and emailing the final recommendation

Each stage includes **code, agent tool calls, and clear markdown explanations** to guide you through the deterministic process.

## A recording of the following notebook can be found at: https://drive.google.com/file/d/1D9eE0zmAheB5F5wQgN9EI0KOce2YURTM/view?usp=sharing

## Initial Setup

In [ ]:
!pip install -r requirements.txt

## Set up OpenAI API Key and Zapier webhook URL

1. **Get your OpenAI API Key:**
   - Go to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys).
   - Log in or sign up for an OpenAI account.
   - Click "Create new secret key" and copy the generated key.

2. **Create a Zapier Webhook URL:**
   - Log in to your [Zapier](https://zapier.com/) account.
   - Click "Create Zap".
   - For the trigger, search for and select **Webhooks by Zapier**.
   - Choose the **Catch Hook** event and continue.
   - Copy the custom webhook URL Zapier provides.
   - Add a "send email" action from Gmail.

3. **Create a `.env` file in your project folder with the following content:**
   ```env
   OPENAI_API_KEY=your-openai-key-here
   MCP_SERVER_URL_SSE=https://hooks.zapier.com/hooks/catch/your_zapier_id/your_path/


## Load in dependencies


In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from agents import RunContextWrapper, function_tool, CodeInterpreterTool, AgentOutputSchema, Agent, Runner

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import get_recommendation
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

# Load Zapier objects
from mcp.zappier_channel import ZapierChannel
import os

## Part 1: Deterministic Workflow – Baseline Agent Flow

For the deterministic flow, the steps of the pipeline are:
1. 🛡️ Check for fraud.
2. ⏱️ Do an SLA check.
3. 💸 Interest rate check.
4. 📝 Make a recommendation for the loan application.

#### 1.1 **Use function tools defined in the **tools/** folder to analyze fraud, SLA and interest rate and provide the Loan Application Journey object as context to be available to the tools**

These function are defined in the following **tools/** files, which you can notice in the panel on the left:
- check_fraud.py
- check_sla.py
- check_acceptability_threshold.py 
- synthesize_summary.py

#### 1.2 **Define fraud_agent, sla_agent, interest_rate_agent and recommendation_agent and provide the PatientJourney object as context to the agents**
The agents are defined in the following Python files in the **agents/** folder:
- fraud_agent.py
- sla_agent.py
- interest_rate_agent.py
- recommendation_agent.py

#### 1.3 **Define the sequence of agents for deterministic flow**

In [3]:
def deterministic_agent_pipeline(test_app):
    """
    Orchestrates the deterministic loan application pipeline:
    1. Runs fraud detection on the application.
    2. Performs SLA compliance check.
    3. Calculates the interest rate.
    4. Generates a recommendation based on the above results.
    """
    # deterministic orchestration of the loan application process
    fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
    sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
    interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))

    # Get the recommendation based on the results of the previous steps
    recommendation = get_recommendation(fraud_result, sla_result, interest_rate_result)
    print("Recommendation:", recommendation)

    # Prepare the context for the report agent
    final_context = {
        "recommendation": recommendation,
        "fraud_result": fraud_result,
        "sla_result": sla_result,
        "interest_rate_result": interest_rate_result
    }

    # Generate the summary report
    report_result = report_agent.tools[0](RunContextWrapper(final_context))
    print("System Summary Report:\n", report_result)
    return report_result



#### 1.4 **Define Sample Data**
#### In the sample data below, one can notice the "KYC indicator". 

KYC stands for Know Your Customer.It is a process used by financial institutions and other regulated companies to:
- Verify the identity of their clients.
- Understand the nature of the customer’s activities (to ensure they're legitimate).
- Assess the risk of illegal intentions, like money laundering, fraud, or terrorist financing.



In [4]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(#
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

#### 1.5 **Execute deterministic flow by providing sample data** 

In [5]:
report_result = deterministic_agent_pipeline(test_app)
report_result

/Users/dariaz/Desktop/Oxford AI Summit - Full code summit assignment/Oxford-Summit-Loan-Application/tools/check_acceptability_threshold.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


Recommendation: **Recommendation: Reject the Loan Application**

While the fraud risk assessment indicates no indicators of fraud and the interest rate results are excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC steps are completed within the configured SLA limits.
System Summary Report:
 📋 Loan Application System Summary
    
This report summarizes the analysis of the loan application system.
Recommendation: **Recommendation: Reject the Loan Application**

While the fraud risk assessment indicates no indicators of fraud and the interest rate results are excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC steps are complet

"📋 Loan Application System Summary\n    \nThis report summarizes the analysis of the loan application system.\nRecommendation: **Recommendation: Reject the Loan Application**\n\nWhile the fraud risk assessment indicates no indicators of fraud and the interest rate results are excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC steps are completed within the configured SLA limits.\nFraud Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}\nSLA Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}\nInterest Rate Result: {'level': 'approve', 'label': 'Excellent'}\n\nRegards,\nTeam 5 - Loan Application Agent\n"

# Part 2: Enhanced Workflow

### 2.2 **Email Report via MCPTool and Zapier Gmail**

This final part demonstrates the **usage of MCP**:
- A report agent is called dynamically to generate a system summary
- The output is sent as an email using `MCPServerSSE` (which calls a Zapier Gmail webhook)

#### 2.1.1 **A function for synthesizing the report is implemented**

In [6]:
@function_tool()
def synthesize_report(wrapper: RunContextWrapper[dict]) -> str:
    context = wrapper.context
    return f"""📋 Loan Application System Summary
Dear {context.get('to', 'Daria Zahaleanu')},
Recommendation: {context.get('recommendation', 'No recommendation')}
Fraud Result: {context.get('fraud_result')}
SLA Result: {context.get('sla_result')}
Interest Rate Result: {context.get('interest_rate_result')}

Regards,
Team 5 - Loan Application Agent
"""

#### 2.1.2 **MCP Implementation**

- mcp/zappier_channel.py : MCPChannel is an abstract base class that defines a common interface for communication channels.
- mcp/mcp.py :This defines a ZapierChannel class that extends a messaging channel interface (MCPChannel) to send messages to a Zapier webhook URL. 

**When the ZapierChannel is called with the send method, it packages the messages into JSON and POSTs them to the configured Zapier webhook, enabling the system to trigger Zapier workflows or integrations based on agent events or outputs.**



In [7]:
#Report to sent out by email
report_result

"📋 Loan Application System Summary\n    \nThis report summarizes the analysis of the loan application system.\nRecommendation: **Recommendation: Reject the Loan Application**\n\nWhile the fraud risk assessment indicates no indicators of fraud and the interest rate results are excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC steps are completed within the configured SLA limits.\nFraud Result: {'flagged': False, 'label': 'No fraud indicators detected', 'risk_score': 0.0, 'triggered_rules': [], 'explanation': 'No indicators present'}\nSLA Result: {'violated': True, 'label': 'SLA violations in: KYC', 'violated_steps': ['KYC'], 'explanation': 'Steps exceeded configured SLA limits.'}\nInterest Rate Result: {'level': 'approve', 'label': 'Excellent'}\n\nRegards,\nTeam 5 - Loan Application Agent\n"

In [8]:
zapier_url = os.getenv("MCP_SERVER_URL_SSE", "https://hooks.zapier.com/hooks/catch/your_zapier_id/your_path/")

zapier_channel = ZapierChannel(webhook_url=zapier_url)

 # Send the report via Zapier
zapier_channel.send(
        message=report_result,
        metadata={"subject": "Loan Application Recommendation", "to": "anadaria.zahaleanu@gmail.com"}
    )

print("✅ Report sent via Zapier email webhook.")


✅ Report sent via Zapier email webhook.


### Further Work: Extending a Deterministic Workflow with the Agents-as-Tools Pattern

An Agents-as-Tools architecture could be applied to the current deterministic flow of the agents by:

- Introducing a central orchestrator agent that supervises the workflow instead of a hardcoded sequence defined in the config.json file
- Letting the orchestrator dynamically choose which agent tools to invoke next based on intermediate outputs (e.g., risk assessment, credit score analysis)
- Supporting conditional branching and iterative refinement where, for example, additional agents (like fraud detection or document verification) can be called only if certain flags are raised.




### Conclusion
The current deterministic approach provides a strong foundation for compliance, auditability, and explainability in loan decisions. 

By clearly documenting each step, tool, and rationale, it makes the entire pipeline understandable and maintainable. 

Additionally, it leverages well-defined agent tools to separate concerns effectively , such as credit checks, risk scoring, and report generation, ensuring modularity and clarity throughout the process.


